<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Try id #259

In [1]:
import sys
sys.path.append('..')

import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [2]:
import taiko as tk
from taiko.play import *
from taiko.performance import *
from taiko.model import *
from taiko.database import *

from sklearn.model_selection import train_test_split
import lightgbm as lgb
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import posixpath
from skimage.io import imshow, imsave, imread

sns.set(font_scale=1.5)

Using TensorFlow backend.


In [3]:
record_df = load_record_df()
record_df = record_df[(record_df['song_id'] >= 1) & (record_df['song_id'] <= 4)]
record_df

,drummer_name,song_id,left_sensor_datetime,right_sensor_datetime,capture_datetime
1,aaaaa,1,L_2018-09-28_111304.csv,R_2018-09-28_111305.csv,capture_2018_09_28_11_13_05
2,aaaaa,2,L_2018-09-28_111610.csv,R_2018-09-28_111611.csv,capture_2018_09_28_11_16_11
3,aaaaa,3,L_2018-09-28_111936.csv,R_2018-09-28_111937.csv,capture_2018_09_28_11_19_37
4,aaaaa,4,L_2018-09-28_112353.csv,R_2018-09-28_112355.csv,capture_2018_09_28_11_23_55
5,aaaaa,1,L_2018-09-28_112912.csv,R_2018-09-28_112913.csv,capture_2018_09_28_11_29_13
6,aaaaa,2,L_2018-09-28_113242.csv,R_2018-09-28_113244.csv,capture_2018_09_28_11_32_44
7,aaaaa,3,L_2018-09-28_113531.csv,R_2018-09-28_113532.csv,capture_2018_09_28_11_35_32
8,aaaaa,4,L_2018-09-28_113909.csv,R_2018-09-28_113910.csv,capture_2018_09_28_11_39_10
9,aaaaa,1,L_2018-09-28_114340.csv,R_2018-09-28_114340.csv,capture_2018_09_28_11_43_41
10,aaaaa,2,L_2018-09-28_114710.csv,R_2018-09-28_114713.csv,capture_2018_09_28_11_47_13


In [4]:
create_play_result()

who_name = aaaaa, song_id = 1, p_order = 1, capture_2018_09_28_11_13_05
result = {'score': 975110, 'good': 81, 'ok': 5, 'bad': 0, 'max_combo': 86, 'drumroll': 78}

who_name = aaaaa, song_id = 2, p_order = 1, capture_2018_09_28_11_16_11
result = {'score': 998510, 'good': 126, 'ok': 1, 'bad': 0, 'max_combo': 127, 'drumroll': 79}

who_name = aaaaa, song_id = 3, p_order = 1, capture_2018_09_28_11_19_37
result = {'score': 995500, 'good': 227, 'ok': 4, 'bad': 0, 'max_combo': 231, 'drumroll': 152}

who_name = aaaaa, song_id = 4, p_order = 1, capture_2018_09_28_11_23_55
result = {'score': 889790, 'good': 215, 'ok': 71, 'bad': 0, 'max_combo': 286, 'drumroll': 301}

who_name = aaaaa, song_id = 1, p_order = 2, capture_2018_09_28_11_29_13
result = {'score': 989360, 'good': 84, 'ok': 2, 'bad': 0, 'max_combo': 86, 'drumroll': 63}

who_name = aaaaa, song_id = 2, p_order = 2, capture_2018_09_28_11_32_44
result = {'score': 987140, 'good': 123, 'ok': 4, 'bad': 0, 'max_combo': 127, 'drumroll': 75}



/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default i

who_name = aaaaa, song_id = 3, p_order = 2, capture_2018_09_28_11_35_32
result = {'score': 989150, 'good': 224, 'ok': 7, 'bad': 0, 'max_combo': 231, 'drumroll': 147}

who_name = aaaaa, song_id = 4, p_order = 2, capture_2018_09_28_11_39_10
result = {'score': 886690, 'good': 217, 'ok': 68, 'bad': 1, 'max_combo': 270, 'drumroll': 275}

who_name = aaaaa, song_id = 1, p_order = 3, capture_2018_09_28_11_43_41
result = {'score': 979660, 'good': 84, 'ok': 2, 'bad': 0, 'max_combo': 86, 'drumroll': 68}

who_name = aaaaa, song_id = 2, p_order = 3, capture_2018_09_28_11_47_13
result = {'score': 993920, 'good': 126, 'ok': 1, 'bad': 0, 'max_combo': 127, 'drumroll': 72}

who_name = aaaaa, song_id = 3, p_order = 3, capture_2018_09_28_11_50_45
result = {'score': 999800, 'good': 229, 'ok': 2, 'bad': 0, 'max_combo': 231, 'drumroll': 152}

who_name = aaaaa, song_id = 4, p_order = 3, capture_2018_09_28_11_54_30
result = {'score': 920320, 'good': 234, 'ok': 52, 'bad': 0, 'max_combo': 286, 'drumroll': 316}



/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default i

who_name = carolyn, song_id = 1, p_order = 1, capture_2018_09_27_15_49_16
result = {'score': 792630, 'good': 54, 'ok': 30, 'bad': 2, 'max_combo': 84, 'drumroll': 42}

who_name = carolyn, song_id = 2, p_order = 1, capture_2018_09_27_15_52_28
result = {'score': 830270, 'good': 95, 'ok': 26, 'bad': 6, 'max_combo': 76, 'drumroll': 59}

who_name = carolyn, song_id = 3, p_order = 1, capture_2018_09_27_15_56_42
result = {'score': 857350, 'good': 170, 'ok': 57, 'bad': 4, 'max_combo': 110, 'drumroll': 90}

who_name = carolyn, song_id = 4, p_order = 1, capture_2018_09_27_16_01_08
result = {'score': 548460, 'good': 92, 'ok': 128, 'bad': 66, 'max_combo': 34, 'drumroll': 186}

who_name = carolyn, song_id = 1, p_order = 2, capture_2018_09_27_16_05_10
result = {'score': 803450, 'good': 57, 'ok': 25, 'bad': 4, 'max_combo': 61, 'drumroll': 48}

who_name = carolyn, song_id = 2, p_order = 2, capture_2018_09_27_16_07_48
result = {'score': 852710, 'good': 95, 'ok': 27, 'bad': 5, 'max_combo': 120, 'drumroll

...

/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default i

who_name = sheep, song_id = 1, p_order = 5, capture_2018_09_29_13_46_22
result = {'score': 987660, 'good': 84, 'ok': 2, 'bad': 0, 'max_combo': 86, 'drumroll': 51}

who_name = sheep, song_id = 2, p_order = 5, capture_2018_09_29_13_49_17
result = {'score': 971480, 'good': 120, 'ok': 7, 'bad': 0, 'max_combo': 127, 'drumroll': 66}

who_name = sheep, song_id = 3, p_order = 5, capture_2018_09_29_13_51_47
result = {'score': 962800, 'good': 213, 'ok': 18, 'bad': 0, 'max_combo': 231, 'drumroll': 108}

who_name = sheep, song_id = 4, p_order = 6, capture_2018_09_29_13_55_12
result = {'score': 815710, 'good': 187, 'ok': 95, 'bad': 4, 'max_combo': 231, 'drumroll': 226}

who_name = sheep, song_id = 1, p_order = 6, capture_2018_09_29_14_04_17
result = {'score': 977960, 'good': 82, 'ok': 4, 'bad': 0, 'max_combo': 86, 'drumroll': 56}

who_name = sheep, song_id = 2, p_order = 6, capture_2018_09_29_14_07_17
result = {'score': 946550, 'good': 115, 'ok': 10, 'bad': 2, 'max_combo': 82, 'drumroll': 67}

who_

/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default i

who_name = sheep, song_id = 2, p_order = 7, capture_2018_09_29_14_20_14
result = {'score': 975270, 'good': 120, 'ok': 7, 'bad': 0, 'max_combo': 127, 'drumroll': 67}

who_name = sheep, song_id = 3, p_order = 7, capture_2018_09_29_14_23_16
result = {'score': 966900, 'good': 215, 'ok': 16, 'bad': 0, 'max_combo': 231, 'drumroll': 112}

who_name = sheep, song_id = 4, p_order = 8, capture_2018_09_29_14_26_42
result = {'score': 876480, 'good': 230, 'ok': 56, 'bad': 0, 'max_combo': 286, 'drumroll': 230}

who_name = sheep, song_id = 1, p_order = 8, capture_2018_09_29_14_30_25
result = {'score': 993610, 'good': 85, 'ok': 1, 'bad': 0, 'max_combo': 86, 'drumroll': 54}

who_name = sheep, song_id = 2, p_order = 8, capture_2018_09_29_14_34_08
result = {'score': 982350, 'good': 122, 'ok': 5, 'bad': 0, 'max_combo': 127, 'drumroll': 66}

who_name = sheep, song_id = 3, p_order = 8, capture_2018_09_29_14_37_22
result = {'score': 956650, 'good': 212, 'ok': 19, 'bad': 0, 'max_combo': 231, 'drumroll': 109}



/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/bin/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default i